# Comparison between fixed/unfixed IQAE vs. Modified IQAE

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from random import sample, seed
from collections import defaultdict

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.algorithms import amplitude_estimators, EstimationProblem
from qiskit.algorithms import IterativeAmplitudeEstimation as BaseIterativeAmplitudeEstimation

from algorithms import IterativeAmplitudeEstimation, ModifiedIterativeAmplitudeEstimation
from algorithms import NoQuantumIterativeAmplitudeEstimation
from operators import *

In [2]:
# for setting a single amplitude, below automates this for arbirtrary numbers of amplitudes
n = 4
N = 2**n
k = N//2
marked = sample(range(N), k)

In [3]:
# Define the estimation problem and oracle function
def make_problem(n, marked):
    
    def good_state(state):
        bin_marked = [(n-len(bin(s))+2)*'0'+bin(s)[2:] for s in marked]
        return (state in bin_marked)

    problem = EstimationProblem(
        state_preparation=A(n),  # A operator
        grover_operator=Q(n, marked),  # Q operator
        objective_qubits=range(n),
        is_good_state=good_state  # the "good" state Psi1 is identified as measuring |1> in qubit 0
    )
    
    return problem

In [4]:
# use local simulator
aer_sim = Aer.get_backend('aer_simulator')
shots = 100

## Compare Modified IQAE to No-Quantum IQAE

In [5]:
# parameters for IQAE

# amplitudes
n = 5
k = 2**n

# epsilons
powers = range(2,7)
epsilons = [(1.0 / (10**power)) for power in powers]

# other
alpha = 0.05
num_trials = 10
confint_method = 'chernoff'
verbose = False

In [ ]:
# stats recording
iae_epsilon = []
iae_nshots = []
miae_epsilon = []
miae_nshots = []

mod_deltas = np.zeros((k+1, len(epsilons)))
iae_deltas = np.zeros((k+1, len(epsilons)))
prob_bug_deltas = np.zeros((k+1, len(epsilons)))
scale_bug_deltas = np.zeros((k+1, len(epsilons)))
both_bug_deltas = np.zeros((k+1, len(epsilons)))

wins = matches = 0

ki = 0
while ki <= k:
    
    failures = 0
    
    # setup problems with specified N,k
    N = 2**n
    marked = sample(range(N), ki)
    problem = make_problem(n, marked)

    # execute experiments for each power
    miae_results = [] 
    iae_results = []

    for i, epsilon in enumerate(epsilons):
        if verbose:
            print('ε:',epsilon)

        mod_IAE = ModifiedIterativeAmplitudeEstimation(epsilon_target=epsilon, 
                                                    alpha=alpha, 
                                                    confint_method=confint_method, 
                                                    quantum_instance=aer_sim)

        IAE = NoQuantumIterativeAmplitudeEstimation(epsilon_target=epsilon, 
                                                    alpha=alpha, 
                                                    confint_method=confint_method, 
                                                    quantum_instance=aer_sim)

        unfixed_IAE = IterativeAmplitudeEstimation(epsilon_target=epsilon, 
                                                    alpha=alpha, 
                                                    confint_method=confint_method, 
                                                    quantum_instance=aer_sim)

        exprs = ['Modified IAE', 'No-Q IAE']

        # configure the number of shots this way and pray that it works
        mod_IAE._quantum_instance._run_config.shots = shots
        IAE._quantum_instance._run_config.shots = shots
        unfixed_IAE._quantum_instance._run_config.shots = shots

        # for recording intermediate algo results
        state = defaultdict(dict)
        state2 = defaultdict(dict)
        state3 = defaultdict(dict)
        
        mod_fails = iae_fails = prob_bug_iae_fails = scale_bug_iae_fails = both_bug_iae_fails = 0
        
        for trial in range(num_trials):

            miae_result = mod_IAE.estimate(problem,
                                        state=state,
                                        verbose=verbose)

            iae_result = IAE.estimate(problem,
                                      state=state2,
                                      k0=ki,
                                      N=N,
                                      verbose=verbose)

            prob_bug_iae_result = unfixed_IAE.estimate(problem,
                                                       state=state,
                                                       bugs={'prob'},
                                                       verbose=verbose)
            
            scale_bug_iae_result = unfixed_IAE.estimate(problem,
                                                        state=state,
                                                        bugs={'scale'},
                                                        verbose=verbose)
            
            both_bug_iae_result = unfixed_IAE.estimate(problem,
                                                       state=state,
                                                       bugs={'prob','scale'},
                                                       verbose=verbose)

            miae_results.append(miae_result)
            iae_results.append(iae_result)
            if verbose:
                print()

            # more than epsilon away
            if miae_result.estimation > ki/N + epsilon or miae_result.estimation < ki/N - epsilon:
                mod_fails += 1
            
            if iae_result.estimation > ki/N + epsilon or iae_result.estimation < ki/N - epsilon:
                iae_fails += 1
            
            if prob_bug_iae_result.estimation > ki/N + epsilon or prob_bug_iae_result.estimation < ki/N - epsilon:
                prob_bug_iae_fails += 1
            
            if scale_bug_iae_result.estimation > ki/N + epsilon or scale_bug_iae_result.estimation < ki/N - epsilon:
                scale_bug_iae_fails += 1
            
            if both_bug_iae_result.estimation > ki/N + epsilon or both_bug_iae_result.estimation < ki/N - epsilon:
                both_bug_iae_fails += 1

            # outside of CI
            if ki/N > miae_result.confidence_interval_processed[1] or ki/N < miae_result.confidence_interval_processed[0]:
                mod_fails += 1
                
            if ki/N > iae_result.confidence_interval_processed[1] or ki/N < iae_result.confidence_interval_processed[0]:
                iae_fails += 1
            
            if ki/N > prob_bug_iae_result.confidence_interval_processed[1] or ki/N < prob_bug_iae_result.confidence_interval_processed[0]:
                prob_bug_iae_fails += 1
            
            if ki/N > scale_bug_iae_result.confidence_interval_processed[1] or ki/N < scale_bug_iae_result.confidence_interval_processed[0]:
                scale_bug_iae_fails += 1
            
            if ki/N > both_bug_iae_result.confidence_interval_processed[1] or ki/N < both_bug_iae_result.confidence_interval_processed[0]:
                both_bug_iae_fails += 1

        # process results
        
        mod_deltas[ki][i] = mod_fails / num_trials
        iae_deltas[ki][i] = iae_fails / num_trials
        prob_bug_deltas[ki][i] = prob_bug_iae_fails / num_trials
        scale_bug_deltas[ki][i] = scale_bug_iae_fails / num_trials
        both_bug_deltas[ki][i] = both_bug_iae_fails / num_trials
        
#         miae_epsilon_i = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in miae_results]
#         miae_nshots_i  = [res.num_oracle_queries for res in miae_results]

#         iae_epsilon_i = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in iae_results]
#         iae_nshots_i  = [res.num_oracle_queries for res in iae_results]

#         print(f'a: {ki}/{N} = {ki/N}')
#         print(f'{exprs[0]} estimations:', [res.estimation for res in miae_results])
#         print(f'{exprs[1]} estimations:', [res.estimation for res in iae_results])

#         miae_total_queries, iae_total_queries = sum(miae_nshots_i), sum(iae_nshots_i)
#         print(f'{exprs[0]} total queries:', miae_total_queries)
#         print(f'{exprs[1]} total queries:', iae_total_queries)

#         print(f'{exprs[0]} epsilons:', miae_epsilon_i)
#         print(f'{exprs[1]} epsilons:', iae_epsilon_i)

#         diff = round(abs(iae_total_queries - miae_total_queries) / miae_total_queries * 100, 2)
#         print('Modified wins?', iae_total_queries < miae_total_queries, f'with {diff}% difference')

#         wins += int(iae_total_queries < miae_total_queries)
#         matches += 1

#         # graph k_i vs num_shots
#         def process_state(state):
#             if len(state) == 0: return [],[]
#             round_shots = state['round_shots']
#             queries = state['n_queries']
#             shots_at_k0 = round_shots.pop(0)
#             queries_at_k0 = queries.pop(0)

#             k_i = [k for k in round_shots]
#             queries_i = [queries[k] for k in k_i]
#             shots_i = [shots_at_k0] + [round_shots[k] for k in k_i]

#             k_i.insert(0, 0.1)

#             return shots_i, queries_i, k_i

#         mod_shots, mod_queries, mod_k = process_state(state)
#         base_shots, base_queries, base_k = process_state(state2)

#         fig,axs = plt.subplots(2,2,figsize=(15,10))

    #     # plot query complexity

    #     axs[0,0].set_yscale('log')
    #     axs[0,0].set_xscale('log')
    #     axs[0,0].set_xlim(1, 10**10)
    #     axs[0,0].set_title('Error vs. Number of Queries')

    #     axs[0,0].scatter(miae_nshots_i, miae_epsilon_i)
    #     axs[0,0].plot(miae_nshots_i, miae_epsilon_i)
    #     axs[0,0].scatter(iae_nshots_i, iae_epsilon_i)
    #     axs[0,0].plot(iae_nshots_i, iae_epsilon_i)

    #     axs[0,0].legend(exprs)

    #     # plots for shots vs k
    #     axs[0,1].set_xscale('log')
    #     axs[0,1].set_title('Shots_i vs. k_i')

    #     axs[0,1].plot(mod_k, mod_shots)
    #     axs[0,1].scatter(mod_k, mod_shots)
    #     axs[0,1].plot(base_k, base_shots)
    #     axs[0,1].scatter(base_k, base_shots)

    #     axs[0,1].legend(exprs)

    #     # plots for nqueries vs k
    #     axs[1,0].set_xscale('log')
    #     axs[1,0].set_yscale('log')
    #     axs[1,0].set_title('Queries_i vs. k_i')

    #     axs[1,0].plot(mod_k[1:], mod_queries)
    #     axs[1,0].scatter(mod_k[1:], mod_queries)
    #     axs[1,0].plot(base_k[1:], base_queries)
    #     axs[1,0].scatter(base_k[1:], base_queries)

    #     axs[1,0].legend(exprs)

    #     # plots for k
    #     axs[1,1].set_yscale('log')
    #     axs[1,1].set_title('k_i vs. i')

    #     axs[1,1].plot(mod_k)
    #     axs[1,1].scatter(range(len(mod_k)), mod_k)
    #     axs[1,1].plot(base_k)
    #     axs[1,1].scatter(range(len(base_k)), base_k)

    #     axs[1,1].legend(exprs)

    #     plt.show()



        # save results for epsilon vs nshots
#         miae_nshots.append(miae_nshots_i)
#         miae_epsilon.append(miae_epsilon_i)
#         iae_nshots.append(iae_nshots_i)
#         iae_epsilon.append(iae_epsilon_i)
    
    ki += 1
    
# iae_nshots = np.array(iae_nshots)
# print('% modified > original:', wins/matches)

In [ ]:
print(f'Modified IAE delta: {mod_deltas.mean()}')
print(f'Bugfixed original IAE delta: {iae_deltas.mean()}')
print(f'Unfixed original IAE delta, probability bug: {prob_bug_deltas.mean()}')
print(f'Unfixed original IAE delta, scaling bug: {scale_bug_deltas.mean()}')
print(f'Unfixed original IAE delta, probability + scaling bugs: {both_bug_deltas.mean()}')

In [ ]:
# for i in range(len(epsilons)):
#     plt.plot(2**np.arange(iae_nshots.shape[0]), iae_nshots[:,i])
#     plt.scatter(2**np.arange(iae_nshots.shape[0]), iae_nshots[:,i])

# plt.title('Number of queries vs. 1/Input amplitude')
# plt.legend(['{:.0e}'.format(eps) for eps in epsilons])
# plt.xscale('log')
# plt.yscale('log')

# plt.show()